# Optimized Inference Benchmark

## Overview
This notebook implements a **high-performance inference benchmark** for the Stage 2 model. Unlike the standard testing pipeline, this script is optimized to approximate real-time deployment conditions.

## Key Optimizations
- **Pre-Allocated GPU Buffers**: Accumulation and weight buffers (`acc_buffer`, `wgt_buffer`) are allocated once on the GPU to prevent memory fragmentation and allocation overhead per frame.
- **GPU-Based Reconstruction**: The Hann-window weighted blending is performed entirely on the GPU using in-place operations (`add_`), significantly reducing CPU-GPU synchronization overhead.
- **Pure Inference Timing**: The benchmark measures the core computational path (Forward Pass + GPU Reconstruction), differentiating it from I/O-bound tasks.
- **Deferred I/O**: Video encoding and image saving occur *after* the benchmark loop to ensure the GPU runs without I/O interruptions.

In [1]:
import json
import sys
import time
from pathlib import Path
from statistics import mean, median
import cv2
import numpy as np
import torch

# Set Project Root
current_dir = Path.cwd()
if current_dir.name == 'testing':
    BASE = current_dir.parent
else:
    BASE = current_dir

sys.path.insert(0, str(BASE))
print(f'Project Root: {BASE}')

from training.helpers.model import MobileNetV3UNetConvLSTMVideo

Project Root: D:\Pycharm Projects\DashcamRaindropsRemoval


## Configuration
- **`TARGET_FPS`**: 33.0 (Real-time target).
- **`ROWS/COLS`**: 3x5 tiling grid.
- **`VIDEO_FPS`**: 10 (Playback speed for the output video).

In [2]:
# --------------------------------------------------------------------
# Paths / Config
# --------------------------------------------------------------------
CHECKPOINT_PATH = BASE / "training" / "checkpoints" / "stage2" / "best_stage2.pth"

RAINY_DIR = BASE / "data" / "data_crapified_test" / "scene_015" / "front-forward"
CLEAN_DIR = BASE / "data" / "data_original" / "scene_015" / "images" / "front-forward"

OUTPUT_DIR = BASE / "testing" / "test_results" / "scene_004_inference_benchmark"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

TARGET_FPS = 33.0
TARGET_MS_PER_FRAME = 1000.0 / TARGET_FPS
VIDEO_FPS = 10
TILE = 512
ROWS, COLS = 3, 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Optimized Helper Functions
- **`load_frame_fullres`**: Loads and normalizes images (CPU).
- **`make_hann_mask`**: Creates the 2D weighting mask. Note the `periodic=False` for correct windowing.
- **`get_tile_coords`**: Computes standard overlapping grid coordinates.

In [3]:
def load_frame_fullres(path: Path) -> torch.Tensor:
    img = cv2.imread(str(path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return torch.from_numpy(img).permute(2, 0, 1).float().div(255.0)


def get_tile_coords(h, w, tile=512, rows=3, cols=5):
    xs = np.linspace(0, w - tile, cols).round().astype(int).tolist()
    ys = np.linspace(0, h - tile, rows).round().astype(int).tolist()
    return [(y, x) for y in ys for x in xs]


def make_hann_mask(tile, device):
    w1 = torch.hann_window(tile, periodic=False, device=device)
    return torch.outer(w1, w1).clamp_min(1e-4).unsqueeze(0)


def chw_to_bgr_uint8(chw: torch.Tensor) -> np.ndarray:
    return cv2.cvtColor((chw.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)

## Main Execution

### Pipeline Steps:
1.  **Setup**: Loads model and pre-calculates tile coordinates.
2.  **Pre-Allocation**: Creates `acc_buffer` and `wgt_buffer` on the GPU *before* the loop. This is critical for performance.
3.  **Benchmark Loop**:
    -   **Load**: Prepare tiles (CPU).
    -   **Transfer**: Move tile batch to GPU.
    -   **Inference**: Run model (`forward`).
    -   **Reconstruct**: Accumulate weighted tiles into the buffers using GPU ops.
    -   **Normalize**: Divide accumulation by weight map.
4.  **Save Results**: After the timing loop is done, iterate through the buffer to save the video and images.

In [4]:
def main():
    print(f"Using device: {device}")
    if not CHECKPOINT_PATH.exists():
         raise FileNotFoundError(f"Checkpoint not found: {CHECKPOINT_PATH}")

    checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
    model = MobileNetV3UNetConvLSTMVideo(hidden_dim=96, out_channels=3).to(device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()

    rainy_files = sorted(RAINY_DIR.glob("*.jpeg"))
    clean_files = sorted(CLEAN_DIR.glob("*.jpeg"))
    n = min(len(rainy_files), len(clean_files))
    if n == 0:
        print("No frames found!")
        return

    # Pre-probe dimensions
    tmp_img = cv2.imread(str(rainy_files[0]))
    h, w, _ = tmp_img.shape
    coords = get_tile_coords(h, w, TILE, ROWS, COLS)
    hann_mask = make_hann_mask(TILE, device)

    # PRE-ALLOCATE GPU BUFFERS (Critical to prevent fragmentation)
    acc_buffer = torch.zeros((3, h, w), device=device)
    wgt_buffer = torch.zeros((1, h, w), device=device)

    times_ms = []
    processed_frames = []

    print(f"\nStarting Optimized Benchmark ({n} frames)...")

    with torch.no_grad():
        for i in range(n):
            # 1. Load data (CPU)
            rainy_chw = load_frame_fullres(rainy_files[i])
            clean_chw = load_frame_fullres(clean_files[i])

            # 2. Build Tile Batch
            tiles = torch.stack([rainy_chw[:, y:y + TILE, x:x + TILE] for y, x in coords]).to(device)

            # --- START TIMING ---
            if device.type == "cuda": torch.cuda.synchronize()
            t0 = time.perf_counter()

            # 3. Model Forward
            out_tiles = model(tiles.unsqueeze(1)).squeeze(1)

            # 4. Weighted Reconstruct (on GPU)
            acc_buffer.zero_()
            wgt_buffer.zero_()
            for idx, (y, x) in enumerate(coords):
                tile_out = out_tiles[idx]
                acc_buffer[:, y:y + TILE, x:x + TILE].add_(tile_out * hann_mask)
                wgt_buffer[:, y:y + TILE, x:x + TILE].add_(hann_mask)

            out_full = acc_buffer / wgt_buffer.clamp_min(1e-6)

            if device.type == "cuda": torch.cuda.synchronize()
            dt_ms = (time.perf_counter() - t0) * 1000.0
            # --- END TIMING ---

            times_ms.append(dt_ms)

            # Post-processing (Moving to CPU to free GPU ASAP)
            out_cpu = out_full.clamp(0, 1).cpu()
            processed_frames.append((rainy_chw, out_cpu, clean_chw, dt_ms))

            if i % 10 == 0:
                print(f"Frame {i}/{n}: {dt_ms:.2f}ms ({1000 / dt_ms:.1f} FPS)")

    # --------------------------------------------------------------------
    # IO Section (Save results after benchmark is done)
    # --------------------------------------------------------------------
    print("\nBenchmark finished. Saving video and images...")
    video_path = OUTPUT_DIR / "comparison_video.mp4"
    writer = cv2.VideoWriter(str(video_path), cv2.VideoWriter_fourcc(*"mp4v"), VIDEO_FPS, (w, h * 3))

    for i, (r_chw, o_chw, c_chw, dt) in enumerate(processed_frames):
        r_bgr = chw_to_bgr_uint8(r_chw)
        o_bgr = chw_to_bgr_uint8(o_chw)
        c_bgr = chw_to_bgr_uint8(c_chw)

        stacked = np.concatenate([r_bgr, o_bgr, c_bgr], axis=0)

        # Overlay labels
        fps = 1000.0 / dt if dt > 0 else 0
        cv2.putText(stacked, f"{dt:.1f}ms | {fps:.1f} FPS", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        writer.write(stacked)
        if i < 10:  # Only save first few as images to save space
            cv2.imwrite(str(OUTPUT_DIR / f"frame_{i:03d}.png"), stacked)

    writer.release()

    # Summary
    avg_ms = mean(times_ms)
    print("\n" + "=" * 30)
    print(f"AVERAGE LATENCY: {avg_ms:.2f} ms")
    print(f"AVERAGE FPS:     {1000 / avg_ms:.2f}")
    print(f"MEETS TARGET:    {avg_ms <= TARGET_MS_PER_FRAME}")
    print("=" * 30)

main()

Using device: cuda


C:\Users\ofiri\AppData\Local\Temp\ipykernel_16444\3285440737.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(CHECKPOINT_PATH, map_location=device

✓ Encoder frozen (weights + BatchNorm running stats)

Starting Optimized Benchmark (200 frames)...
Frame 0/200: 534.09ms (1.9 FPS)
Frame 10/200: 66.41ms (15.1 FPS)
Frame 20/200: 71.09ms (14.1 FPS)
Frame 30/200: 65.77ms (15.2 FPS)
Frame 40/200: 66.92ms (14.9 FPS)
Frame 50/200: 67.31ms (14.9 FPS)
Frame 60/200: 67.12ms (14.9 FPS)
Frame 70/200: 66.71ms (15.0 FPS)
Frame 80/200: 72.08ms (13.9 FPS)
Frame 90/200: 72.99ms (13.7 FPS)
Frame 100/200: 71.84ms (13.9 FPS)
Frame 110/200: 70.00ms (14.3 FPS)
Frame 120/200: 78.87ms (12.7 FPS)
Frame 130/200: 231.76ms (4.3 FPS)
Frame 140/200: 70.42ms (14.2 FPS)
Frame 150/200: 71.75ms (13.9 FPS)
Frame 160/200: 99.74ms (10.0 FPS)
Frame 170/200: 79.78ms (12.5 FPS)
Frame 180/200: 69.37ms (14.4 FPS)
Frame 190/200: 72.91ms (13.7 FPS)

Benchmark finished. Saving video and images...



KeyboardInterrupt

